In [ ]:
import pandas as pd
import requests
import sqlite3
import os
import time

In [ ]:
# Caminho do DB
db_path = os.path.abspath("../datasets/data.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Pipeline de armazenamento referente aos deputados 

## Tabela Deputados

In [ ]:
conn = sqlite3.connect(db_path)

cursor = conn.cursor()

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS deputados_56 (
        id INTEGER NOT NULL PRIMARY KEY,
        nome TEXT NOT NULL,
        siglaUf TEXT NOT NULL,
        siglaPartido TEXT NOT NULL,
        urlFoto TEXT NOT NULL,
        uri TEXT NOT NULL     
    );
    """
)

conn.commit()
conn.close()

In [ ]:
base_url = "https://dadosabertos.camara.leg.br/api/v2/deputados"
params = {
    "dataInicio": "2022-01-01",
    "dataFim": "2022-12-31",
    "ordem": "ASC",
    "ordenarPor": "siglaUF"
}


try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    if "dados" in data and data["dados"]:
        deputados = pd.DataFrame(data["dados"])
        deputados = deputados[["id", "nome", "siglaUf", "siglaPartido", "urlFoto", "uri"]]

        with sqlite3.connect(db_path) as conn:
            deputados.to_sql("deputados_56", conn, if_exists="replace", index=False)
            conn.execute("CREATE INDEX IF NOT EXISTS idx_deputados_id ON deputados_56 (id)")

    else:
        print("Nenhum dado encontrado.")

    url = None
    if "links" in data:
        for link in data["links"]:
            if link.get("rel") == "next":
                url = link.get("href")
                break

except requests.RequestException as e:
    print(f"Erro na requisição: {e}")

## Tabela Deputado - Detalhes

In [ ]:
# Tabela detalhes
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS deputados_56_detalhes (
        id INTEGER NOT NULL PRIMARY KEY,
        nomeCivil TEXT NOT NULL,
        cpf TEXT NOT NULL,
        dataNascimento TEXT,
        dataFalecimento TEXT,
        escolaridade TEXT,
        profissoes TEXT, 
        redeSocial TEXT
    );
    """
)

conn.commit()
conn.close()

In [ ]:
# Função para converter listas de redes sociais para string
def convert_rede_social(rede_social):
    if isinstance(rede_social, list):
        return '; '.join(rede_social)
    return rede_social

In [ ]:
try:
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id, uri FROM deputados_56")
        uris = cursor.fetchall()

        for id, uri in uris:
            response = requests.get(uri)
            response.raise_for_status()
            data = response.json()
                

            if "dados" in data and data["dados"]:
                dados = data["dados"]
                detalhes = {
                    "id": id,
                    "nomeCivil": dados.get("nomeCivil", ""),
                    "cpf": dados.get("cpf", ""),
                    "dataNascimento": dados.get("dataNascimento", ""),
                    "dataFalecimento": dados.get("dataFalecimento", ""),
                    "escolaridade": dados.get("escolaridade", ""),
                    "profissoes": dados.get("profissoes", ""),
                    "redeSocial": convert_rede_social(dados.get("redeSocial", ""))
                }
                
                
                
                # Verificando se o deputado já está na tabela
                cursor.execute("SELECT id FROM deputados_56_detalhes WHERE id = ?", (id,))
                result = cursor.fetchone()
                
                
                if not result:
                    cursor.execute(
                        """
                        INSERT INTO deputados_56_detalhes (id, nomeCivil, cpf, dataNascimento, dataFalecimento, escolaridade, profissoes, redeSocial)
                        VALUES (:id, :nomeCivil, :cpf, :dataNascimento, :dataFalecimento, :escolaridade, :profissoes, :redeSocial)
                        """,
                        detalhes
                    )
                    conn.commit()
            
                time.sleep(0.5)
                print("Deputado inserido com sucesso:", id)

            else:
                print("Nenhum dado encontrado para a URI:", uri)

except requests.RequestException as e:
    print(f"Erro na requisição: {e}")

except sqlite3.Error as e:
    print(f"Erro no banco de dados: {e}")

### Inserindo profissão de cada Deputado

In [ ]:
with sqlite3.connect(db_path) as conn:
    for id, uri in uris:
        try:
            # Fazendo a requisição para obter as profissões
            response = requests.get(f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/profissoes")
            response.raise_for_status()
            data = response.json()
                            
            if "dados" in data and data["dados"]:
                profissao = data["dados"][0]["titulo"]  # Obtendo o título da profissão
                        
                conn.execute("UPDATE deputados_56_detalhes SET profissoes = ? WHERE id = ?", (profissao, id))
                conn.commit()
                
                time.sleep(0.5)
                print("Profissão inserida com sucesso:", id, profissao)
            
        except requests.RequestException as e:
            print(f"Erro na requisição: {e}") 

## Detalhes do partido referente ao(s) Deputado(s)

In [ ]:
# Conexão com data.db
conn = sqlite3.connect(db_path)

# Cursor
cursor = conn.cursor()

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS partidos (
        id INTEGER NOT NULL PRIMARY KEY,
        sigla TEXT NOT NULL,
        nome TEXT NOT NULL,
        urlLogo TEXT,
        uri TEXT NOT NULL
    );
    """
)

conn.commit()
conn.close()

In [ ]:
url = "https://dadosabertos.camara.leg.br/api/v2/partidos?dataInicio=2022-01-01&dataFim=2022-12-31&ordem=ASC&ordenarPor=sigla"

# Lista para armazenar os dados de todos os partidos
todos_partidos = []

while url:  # Continua enquanto houver uma URL válida
    response = requests.get(url)

    if response.status_code == 200:
        # Converte a resposta para JSON
        data = response.json()
        
        # Extraindo os dados corretamente (data["dados"] é uma lista)
        for partido in data["dados"]:
            partido_info = {
                "id": partido.get("id", ""),
                "sigla": partido.get("sigla", ""),
                "nome": partido.get("nome", ""),
                "urlLogo": partido.get("urlLogo", ""),
                "uri": partido.get("uri", "")
            }
            todos_partidos.append(partido_info)

        # Verifica se há próxima página
        url = None  # Assume que não há próxima página
        for link in data["links"]:  # Os links estão na resposta JSON
            if link.get("rel") == "next":  # Se houver link "next", continue extraindo dados
                url = link.get("href")
                break  # Sai do loop ao encontrar a próxima URL

        # Tempo de espera entre requisições para evitar sobrecarga na API
        time.sleep(0.5)

    else:
        print(f"Erro na requisição ({response.status_code})")
        print("Saindo do loop...")
        break  # Interrompe o loop se houver erro na requisição

# Criando o DataFrame corretamente a partir da lista de dicionários
if todos_partidos:
    df_final = pd.DataFrame(todos_partidos)

    # Salvando no banco de dados SQLite
    with sqlite3.connect(db_path) as conn:
        df_final.to_sql("partidos", conn, if_exists="replace", index=False)

    print("✅ Dados dos partidos armazenados com sucesso no banco de dados!")
else:
    print("❌ Nenhum dado foi coletado.")

In [ ]:
# Consultando uri dos partidos e atualizando urlLogo
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT id, uri FROM partidos")
partidos = cursor.fetchall()

for id, uri in partidos:
    response = requests.get(uri)
    response.raise_for_status()
    data = response.json()

    if "dados" in data and data["dados"]:
        partido = data["dados"]
        url_logo = partido.get("urlLogo", "")

        cursor.execute("UPDATE partidos SET urlLogo = ? WHERE id = ?", (url_logo, id))
        conn.commit()

        print("URL da logo atualizada com sucesso:", id)

    else:
        print("Nenhum dado encontrado para a URI:", uri)

## Tabela Despesas

In [ ]:
# Conexão com data.db
conn = sqlite3.connect(db_path)

# Cursor
cursor = conn.cursor()

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS despesas (
        id_deputado INTEGER NOT NULL,
        ano INTEGER NOT NULL,
        mes INTEGER NOT NULL,
        tipoDespesa TEXT NOT NULL,
        codDocumento INTEGER NOT NULL,
        tipoDocumento TEXT NOT NULL,
        codTipoDocumento INTEGER NOT NULL,
        dataDocumento TEXT NOT NULL,
        numDocumento TEXT NOT NULL,
        valorDocumento REAL NOT NULL,
        valorLiquido REAL NOT NULL,
        urlDocumento TEXT,
        nomeFornecedor TEXT NOT NULL,
        cnpjCpfFornecedor TEXT NOT NULL,
        codLote INTEGER NOT NULL,
        FOREIGN KEY (id_deputado) REFERENCES deputados(id)
    );
    """
)

conn.commit()
conn.close()


In [ ]:
with sqlite3.connect(db_path) as conn:
    deputados = pd.read_sql_query("SELECT id FROM deputados_56", conn)

deputados_ids = deputados["id"].unique()

# Lista para armazenar falhas na requisição
despesas_nao_encontradas = []

# Loop para buscar despesas de cada deputado
for deputado in deputados_ids:
    url_despesas = (
        f"https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado}/despesas?"
        f"idLegislatura=56&ano=2022&itens=100&ordem=ASC&ordenarPor=dataDocumento"
    )
    
    while url_despesas:  # Enquanto houver páginas com dados
        try:
            response = requests.get(url_despesas)
            response.raise_for_status()
            data = response.json()
            
            if "dados" in data and data["dados"]:  
                despesas = pd.DataFrame(data["dados"])

                # Mantemos apenas as colunas relevantes, se os dados existem
                if not despesas.empty:
                    despesas["id_deputado"] = deputado  # Adicionamos o ID do deputado
                    despesas = despesas[[
                        "id_deputado", "ano", "mes", "tipoDespesa", "codDocumento", "tipoDocumento",
                        "codTipoDocumento", "dataDocumento", "numDocumento", "valorDocumento",
                        "valorLiquido", "urlDocumento", "nomeFornecedor", "cnpjCpfFornecedor", "codLote"
                    ]]

                    # Salvando no banco de dados
                    with sqlite3.connect(db_path) as conn:
                        despesas.to_sql("despesas", conn, if_exists="append", index=False)
                        
                    time.sleep(0.5)  # Aguarda 0.5 segundos para a próxima requisição

            else:
                # Se a API retorna um bloco vazio, saímos do loop
                break

            # Paginação: obtendo o link da próxima página
            url_despesas = None  # Assume como padrão que não há mais páginas
            for link in data.get("links", []):
                if link.get("rel") == "next":
                    url_despesas = link.get("href")
                    break  # Encontrou a próxima página, sai do loop

        except requests.RequestException as e:
            print(f"Erro de requisição para deputado {deputado}: {e}")
            despesas_nao_encontradas.append(deputado)
            break
        except Exception as e:
            print(f"Erro inesperado ao processar deputado {deputado}: {e}")
            despesas_nao_encontradas.append(deputado)
            break

# Exibindo os deputados que não tiveram suas despesas encontradas
if despesas_nao_encontradas:
    print("Deputados com falha na requisição de despesas:", despesas_nao_encontradas)